# Modelling with-in cell heterogeneity with satellite imagery

## Abstract

There is a growing scientific literature on using mobile phone metadata, especially CDRs as auxiliary data for official statistics. When an individual makes a call, sends a message or uses the mobile internet, meta information about this interaction, such as the timestamp and the location, are stored in the database of the mobile network operator. Researchers exploit those spatio-temporal references for geo-located analysis. Although the spatio-temporal element in mobile phone metadata is a key argument for using MPD, little research has been done on the precision of the location information. In the case of CDRs, the geographic link is provided by the antenna location, often stored as a point coordinate of the physical location of the corresponding antenna. Due it's simplicity, large parts of the scientific literature treat them as such. However, the interactions captured by the antenna do not happen entirely at this exact coordinate, but within the coverage area of the antenna - the cell. Another widely used attempt to account for the coverage area is voronoi tessellation - dividing the space around antennas depending on the distance to the surrounding antennas. This - at best - represents a naive approximation of the true coverage area as it does not take overlapping coverage areas, areas without coverage and heterogeneity within the cells into account. This paper will build on recent results from Ricchiato et al. on Bayesian estimation of coverage areas and extends it satellite imagery to account for within-cell heterogeneity of interactions.

## Preparations

#### Modules

In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import rasterio
import rasterio.mask
from rasterio.windows import Window
import folium
import math
import numpy as np
import itertools
import gdal
import multiprocessing as mp
from rasterstats import point_query
import time
import shapely
import scipy.spatial as spatial
from geovoronoi import voronoi_regions_from_coords
from tqdm import tqdm
import gc
from math import radians, cos, sin, asin, sqrt
import weightedstats as ws
import robustats as rs

#### Paths & Files

In [ ]:
file_antenna = './midsave/SITE_ARR_LONLAT_EXACT.csv'
file_commune_map = './midsave/shape_com.shp'
file_guf = './input/senegal.tif' # "sen_bsgmi_v0a_100m_2013.tif"
file_wpg = './input/sen_ppp_2013.tif'
file_cdr = './midsave/NUTS5_tower.csv'
file_bandicoot = './midsave/bandicoot_tower.csv'
file_map_pixel = './midsave/map_pixel.shp'
file_map_grid = './midsave/map_grid.shp'
file_map_pixel_knn = './midsave/w_knn.csv'
file_map_grid_knn = './midsave/w_grid_knn.csv'
file_map_pixel_adm = './midsave/w_knn_adm.csv'
file_map_grid_adm = './midsave/w_grid_adm.csv'
file_map_voronoi = './midsave/map_voronoi.csv'

#### Notebook options

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### Functions

In [ ]:
%%capture
# load census and projections data
%run ./helpers/functions.ipynb

## Data

#### Map

In [ ]:
map_commune = gpd.read_file(file_commune_map).rename(columns={"SP_ID": "MAP_ID", "Shape_Area": "SUPERFICIE", 'CCOD_CRCA' : 'COD_ENTITE'})
map_commune.crs = map_commune.to_crs({'init': 'epsg:4326'})
map_commune.loc[(map_commune.COD_ENTITE == '02120201') & (map_commune.COD_CRCA == '05'), 'COD_ENTITE'] = '02120205'
map_commune.loc[(map_commune.COD_ENTITE == '02220101') & (map_commune.CCOD_CAV == '022202'), 'COD_ENTITE'] = '02220201'

In [ ]:
map_commune['MAP_ID'] = map_commune.MAP_ID.astype('uint16')

In [ ]:
map_commune.to_file('./midsave/map_commune.shp')

#### Antenna locations

In [ ]:
ant_points = pd.read_csv(file_antenna, sep = ';', header = None, names = ('site_id', 'lon', 'lat'))

In [ ]:
ant_points.loc[ant_points.site_id == 254, 'lon'] = -17.433461
ant_points.loc[ant_points.site_id == 254, 'lat'] = 14.660496

In [ ]:
ant_points.head(3)

In [ ]:
map_ant_points = gpd.GeoDataFrame(ant_points.drop(['lon', 'lat'], axis=1),
                    crs=map_commune.crs,
                    geometry=[Point(xy) for xy in zip(ant_points.lon, ant_points.lat)])

In [ ]:
print('Site locations')
map_ant_points.plot()

#### RGPHAE 2013

## Methodology

#### Point allocation

In [ ]:
map_intersection_points = gpd.tools.sjoin(map_ant_points, map_commune, how="left", op='intersects')

In [ ]:
map_intersection_points.head(3)

In [ ]:
xwalk_points = map_intersection_points[['site_id', 'MAP_ID']].copy()

In [ ]:
xwalk_points.head(3)

How many communes do not have an antenna site located within?

In [ ]:
len(map_commune.MAP_ID.unique())-len(xwalk_points.MAP_ID.drop_duplicates())

How many sites have not been allocated to a commune?

In [ ]:
pd.isna(xwalk_points.MAP_ID).sum()

With the point allocation of sites to communes, 46 communes do not have any mobile phone metadata covariates.

### Calculating site density

In [ ]:
map_intersection_points.head(3)

In [ ]:
counts = map_intersection_points.groupby('MAP_ID')['site_id'].count()

In [ ]:
map_urbanity = map_commune.merge(counts, how = 'right', on = 'MAP_ID')

In [ ]:
map_urbanity['site_density'] = map_urbanity.site_id/map_urbanity.SUPERFICIE

In [ ]:
map_urbanity.to_file('./midsave/map_urbanity.shp')

# Voronoi tessellation

Run script

In [ ]:
%%capture
%run ./helpers/voronoi_tessellation.ipynb

In [ ]:
map_intersection_voronoi.describe()

# Global Urban Footprint

There are two different approaches to combine GUF data with the other information:

1. Count the number of black (i.e. "urban") pixels per area and calculate spatial weights from it. For antenna areas, this would be a weight to distribute the total number of home_users incl. their attributes to the respective administrative area while assuming equal probabilities among intersecting antennas. For administrative areas, this would be a weight to distribute the total number of inhabitants incl. their attributes to the respective antenna. So we could calculate a calibration weight for every area in the map_intersection_circle file.

2. Calculate the centroid of each black pixel in Senegal, assign it to an administrative area, draw a buffer around it and calculate the value of the log-normal pdf from each antenna within the buffer. The antenna with the highest value gets the black pixel assigned. Both antenna and area values are then uniformly distributed across the black pixels. and map it to the 10 nearest antennas (nearest neighbor) and the respective administrative area (point-in-polygon). The pixel is then allocated to the antenna with strongest signal (approximated via a poisson point process). This would allow for a distribution of attributes by pixel (~12m).

### Counting black pixels under unequal probabilities - by pixel

In [ ]:
with rasterio.open(file_guf) as src:
    print(src.profile)

Extract the centroid of each pixel

In [ ]:
ds = gdal.Open(file_guf)
gt = ds.GetGeoTransform()

In [ ]:
results = []

if __name__ == "__main__":
    start_time = time.time()  
    
    # Repeats the compute intensive operation on 10 data frames concurrently
    pool = mp.Pool(processes=mp.cpu_count()-1)
    [pool.apply_async(extract_pixel_information, args=(file_guf, x, ds.RasterYSize, 100), callback=collect_results) for x in range(0, ds.RasterXSize, 100)]
    pool.close()
    pool.join()
    
    # Converts list of lists to a data frame
    map_pixel = gpd.GeoDataFrame(results, columns = ['geometry', 'value'], crs = map_commune.crs)
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
map_pixel.head(3)

Creating pixel id

In [ ]:
map_pixel.insert(loc = 0, column='pixel_id', value=list(range(len(map_pixel))))

In [ ]:
map_pixel.head(3)

In [ ]:
map_pixel[['pixel_id', 'geometry']].to_file('./midsave/map_pixel.shp')

### Technical antenna specifications

In [ ]:
ant_details = map_intersection_points[['site_id', 'geometry', 'MAP_ID']].merge(map_urbanity[['MAP_ID', 'site_density']], on = 'MAP_ID', how = 'left')

In [ ]:
ant_details['antenna_height'] = 30

In [ ]:
ant_details['settlement_type'] = 'rural'
ant_details.loc[ant_details.site_density >= ant_details.site_density.median(), 'settlement_type'] = 'suburban' # .quantile(0.25)
ant_details.loc[ant_details.site_density >= 1, 'settlement_type'] = 'urban'

In [ ]:
ant_details.loc[ant_details.settlement_type == 'rural', 'frequency'] = np.random.RandomState().binomial(
    1, 0.0, ant_details.loc[ant_details.settlement_type == 'rural'].shape[0])
ant_details.loc[ant_details.settlement_type == 'suburban', 'frequency'] = np.random.RandomState().binomial(
    1, 0.7, ant_details.loc[ant_details.settlement_type == 'suburban'].shape[0])
ant_details.loc[ant_details.settlement_type == 'urban', 'frequency'] = np.random.RandomState().binomial(
    1, 0.9, ant_details.loc[ant_details.settlement_type == 'urban'].shape[0])

In [ ]:
ant_details.loc[ant_details['frequency'] == 0, 'frequency'] = 900
ant_details.loc[ant_details['frequency'] == 1, 'frequency'] = 2100

In [ ]:
ant_details.to_file('./midsave/ant_details.shp')

In [ ]:
ant_details[ant_details.frequency == 2100].plot()

### 10-Nearest Neighbors Pixel and Antennas

Load the extended HATA model -> https://ecocfl.cept.org/display/SH/A17.3.1+Outdoor-outdoor+propagation

Assumptions:

- 2100 MHz in big urban areas (tbd), 900 MHz otherwise
- Only Dakar urban area, all others suburban or rural
- nlos in Dakar, los in suburban/ rural areas
- all outdoor
- all above roof
- all at 30m

In [ ]:
%%capture
# load census and projections data
%run ./helpers/propagation_model_extended_hata.ipynb

Load pixel data

In [ ]:
map_pixel = gpd.read_file(file_map_pixel)

Convert CRS to unit in meters

In [ ]:
map_pixel = map_pixel.to_crs({'init': 'epsg:31028'})

In [ ]:
map_ant_points = map_ant_points.to_crs({'init': 'epsg:31028'})

Run the nearest neighbor search and calculate the path loss from the extracted distance

In [ ]:
nA = np.array(list(zip(map_pixel.geometry.x, map_pixel.geometry.y)) )
nB = np.array(list(zip(map_ant_points.geometry.x, map_ant_points.geometry.y)) )
btree = spatial.cKDTree(nB)

In [ ]:
map_pixel_knn = pd.DataFrame(columns = ['pixel_id', 'site_id', 'path_loss'])

for i in tqdm(range(1,6)):
    
    # Calculate the k-nearest antennas to each pixel
    dist, idx = btree.query(nA, k = [i])
    temp = pd.concat(
        [map_pixel[['pixel_id']], map_ant_points.loc[idx.flatten(), map_ant_points.columns != 'geometry'].reset_index(),
         pd.Series(dist.flatten(), name = 'distance') / 10**3], axis = 1).drop(columns = ['index'])
    
    # Merge technical specifications of the antennas
    temp = temp.merge(
        ant_details[['site_id', 'antenna_height', 'settlement_type', 'frequency']], on = 'site_id', how = 'left').astype(
        {'pixel_id': 'uint32', 'site_id': 'uint16', 'antenna_height': 'uint8', 'frequency': 'uint16'})
    
    # Calculate the path loss between each pixel and antenna
    temp['alpha'] = temp.apply(lambda row: alpha_value(row['frequency'], row['distance'], row['antenna_height']), axis = 1)
    temp['path_loss'] = temp.apply(lambda row: calculate_path_loss_hata_novar(row['frequency'], row['settlement_type'], row['distance'], row['antenna_height'], 1, row['alpha']), axis = 1)

    map_pixel_knn = map_pixel_knn.append(temp[['pixel_id', 'site_id', 'path_loss']])
    
    gc.collect()

In [ ]:
btree = None
nA = None
nB = None
dist = None
idx = None

In [ ]:
gc.collect()

In [ ]:
map_pixel_knn.head(3)

In [ ]:
map_pixel_knn.to_csv('./midsave/map_pixel_knn.csv', index = False)

### Point-to-polygon allocation of pixels to administrative areas

In [ ]:
map_pixel = gpd.read_file(file_map_pixel)

In [ ]:
map_pixel_adm = gpd.tools.sjoin(map_pixel, map_commune[['MAP_ID', 'geometry']], how="left", op='intersects').drop(columns=['index_right'])

In [ ]:
len(map_pixel_adm.MAP_ID.unique())

In [ ]:
gc.collect()

Some pixels have been allocated to more than one commune

In [ ]:
map_pixel_adm.drop_duplicates(subset = 'pixel_id', keep = False, inplace = True)

Some pixels lay outside the country borders. Drop them.

In [ ]:
len(map_pixel_adm)

In [ ]:
map_pixel_adm = map_pixel_adm.dropna()

In [ ]:
len(map_pixel_adm)

In [ ]:
len(map_pixel_adm.MAP_ID.unique())

In [ ]:
map_pixel_adm.set_index('MAP_ID', inplace = True)
map_pixel_adm['w_knn_adm'] = 1 / map_pixel_adm.groupby('MAP_ID')['pixel_id'].count()
map_pixel_adm.reset_index(inplace = True)

In [ ]:
map_pixel_adm[['MAP_ID', 'pixel_id', 'w_knn_adm']].to_csv('./midsave/w_knn_adm.csv', index = False)

# Calculate spatial weights from the inverse signal strength

Load SIM card counts of the full year of 2013

In [ ]:
map_pixel_adm = pd.read_csv('./midsave/w_knn_adm.csv').astype(
        {'MAP_ID': 'uint16', 'pixel_id': 'uint32', 'w_knn_adm': 'float64'})

In [ ]:
map_pixel_knn = pd.read_csv('./midsave/map_pixel_knn.csv').astype(
        {'pixel_id': 'uint32', 'site_id': 'uint16', 'path_loss': 'float32'})

In [ ]:
map_pixel_knn = map_pixel_knn[map_pixel_knn.pixel_id.isin(map_pixel_adm.pixel_id) == True]

In [ ]:
map_pixel_knn['received_signal_strength'] = 45 - map_pixel_knn.path_loss

In [ ]:
map_pixel_knn.head(3)

Dropping all connections that are below RSSI -110

In [ ]:
map_pixel_knn = map_pixel_knn[map_pixel_knn.received_signal_strength > -110]

In [ ]:
map_pixel_knn.shape

Implementing winner-takes-it-all aka. best server approach

In [ ]:
map_pixel_knn.set_index('pixel_id', inplace = True)
map_pixel_knn['w_best_ant'] = 0
map_pixel_knn.loc[map_pixel_knn.groupby('pixel_id')['received_signal_strength'].transform(max) == map_pixel_knn.received_signal_strength, 'w_best_ant'] = 1
map_pixel_knn.reset_index(inplace=True)

In [ ]:
df = map_pixel_knn.groupby('pixel_id')['w_best_ant'].sum().reset_index()

for i in df[df.w_best_ant > 1].pixel_id:
    
    if map_pixel_knn[map_pixel_knn.pixel_id == i].w_best_ant.sum() > 1:
        
        map_pixel_knn.loc[map_pixel_knn.pixel_id == i, 'w_best_ant'] = np.where(
            map_pixel_knn[map_pixel_knn.pixel_id == i].w_best_ant == 1, 1/map_pixel_knn[map_pixel_knn.pixel_id == i].w_best_ant.sum(), 0)
    
    else:
        continue

In [ ]:
map_pixel_knn.tail(3)

In [ ]:
map_pixel_knn.set_index('site_id', inplace = True)
map_pixel_knn['w_best_ant_pop'] = map_pixel_knn.w_best_ant
map_pixel_knn['w_best_ant_sum'] = map_pixel_knn.groupby('site_id')['w_best_ant_pop'].sum()
map_pixel_knn['w_best_site'] = map_pixel_knn.w_best_ant_pop / map_pixel_knn.w_best_ant_sum # might cause issues when w_best_ant_sum is 0
map_pixel_knn.reset_index(inplace=True)

In [ ]:
map_pixel_knn.groupby('site_id')['w_best_site'].sum().describe()

In [ ]:
map_pixel_knn = map_pixel_knn.drop(columns = {'w_best_ant_pop', 'w_best_ant_sum'})

In [ ]:
gc.collect()

Calculating inverse distance weights based on RSS measures.

In [ ]:
map_pixel_knn['received_signal_strength_inverted'] = 1 / (map_pixel_knn.received_signal_strength)**2

In [ ]:
map_pixel_knn.set_index('pixel_id', inplace = True)
map_pixel_knn['rss_sum_inv'] = map_pixel_knn.groupby('pixel_id')['received_signal_strength_inverted'].sum()
map_pixel_knn['w_knn_ant'] = map_pixel_knn.received_signal_strength_inverted / map_pixel_knn.rss_sum_inv
map_pixel_knn.reset_index(inplace=True)

In [ ]:
map_pixel_knn.set_index('site_id', inplace = True)
map_pixel_knn['w_knn_ant_pop'] = map_pixel_knn.w_knn_ant
map_pixel_knn['w_knn_ant_sum'] = map_pixel_knn.groupby('site_id')['w_knn_ant_pop'].sum()
map_pixel_knn['w_knn_site'] = map_pixel_knn.w_knn_ant_pop / map_pixel_knn.w_knn_ant_sum
map_pixel_knn.reset_index(inplace=True)

In [ ]:
map_pixel_knn = map_pixel_knn.drop(columns = {
    'received_signal_strength', 'received_signal_strength_inverted', 'rss_sum_inv', 'w_knn_ant', 'w_knn_ant_pop', 'w_knn_ant_sum'})

In [ ]:
gc.collect()

In [ ]:
map_pixel_knn[['pixel_id', 'site_id', 'path_loss', 'w_best_site', 'w_knn_site']].to_csv('./midsave/w_knn.csv', index = False)

# World Pop Grid

Here I somehow need to introduce the World Pop grid!!! -> https://www.worldpop.org/geodata/summary?id=4683

In [ ]:
with rasterio.open(file_wpg) as src:
    print(src.profile)

In [ ]:
ds = gdal.Open(file_wpg)
gt = ds.GetGeoTransform()

In [ ]:
results = []

if __name__ == "__main__":
    start_time = time.time()  
    
    pool = mp.Pool(processes=mp.cpu_count()-1)
    [pool.apply_async(extract_grid_information, args=(file_wpg, x, ds.RasterYSize, 100), callback=collect_results) for x in range(0, ds.RasterXSize, 100)]
    pool.close()
    pool.join()
    
    map_grid = gpd.GeoDataFrame(results, columns = ['geometry', 'pop_per_grid'], crs = map_commune.crs).astype({'pop_per_grid' : 'float32'})
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
map_grid.head(3)

In [ ]:
map_grid.shape

In [ ]:
map_grid.dtypes

Create grid id

In [ ]:
map_grid.insert(loc = 0, column='grid_id', value=list(range(len(map_grid))))

In [ ]:
map_grid.dtypes

In [ ]:
map_grid = gpd.GeoDataFrame(map_grid, crs=map_commune.crs)

In [ ]:
map_grid[['grid_id', 'geometry']].to_file('./midsave/map_grid_nopop.shp')

In [ ]:
map_grid[['grid_id', 'pop_per_grid']].to_csv('./midsave/map_grid_nogeo.csv')

In [ ]:
map_grid[['grid_id', 'pop_per_grid', 'geometry']].to_file('./midsave/map_grid.shp')

### 10-Nearest Neighbors Grid and Antennas

In [ ]:
map_grid = gpd.read_file('./midsave/map_grid_nopop.shp')

In [ ]:
map_grid.dtypes

In [ ]:
map_grid = map_grid.to_crs({'init': 'epsg:31028'})

In [ ]:
nA = np.array(list(zip(map_grid.geometry.x, map_grid.geometry.y)))

In [ ]:
map_grid = map_grid.drop(columns = ['geometry'])

In [ ]:
map_ant_points = map_ant_points.to_crs({'init': 'epsg:31028'})

In [ ]:
nB = np.array(list(zip(map_ant_points.geometry.x, map_ant_points.geometry.y)))
btree = spatial.cKDTree(nB)

In [ ]:
map_grid_knn = pd.DataFrame(columns = ['grid_id', 'site_id', 'path_loss'])

for i in tqdm(range(1,6)):
    
    # Calculate the k-nearest antennas to each pixel
    dist, idx = btree.query(nA, k = [i])
    temp = pd.concat(
        [map_grid[['grid_id']], map_ant_points.loc[idx.flatten(), map_ant_points.columns != 'geometry'].reset_index(),
         pd.Series(dist.flatten(), name = 'distance') / 10**3], axis = 1).drop(columns = ['index'])
    
    # Merge technical specifications of the antennas
    temp = temp.merge(
        ant_details[['site_id', 'antenna_height', 'settlement_type', 'frequency']], on = 'site_id', how = 'left').astype(
        {'grid_id': 'uint32', 'site_id': 'uint16', 'antenna_height': 'uint8', 'frequency': 'uint16'})
    
    # Calculate the path loss between each pixel and antenna
    temp['alpha'] = temp.apply(lambda row: alpha_value(row['frequency'], row['distance'], row['antenna_height']), axis = 1)
    temp['path_loss'] = temp.apply(lambda row: calculate_path_loss_hata_novar(row['frequency'], row['settlement_type'], row['distance'], row['antenna_height'], 1, row['alpha']), axis = 1)

    map_grid_knn = map_grid_knn.append(temp[['grid_id', 'site_id', 'path_loss']])
    
    gc.collect()

In [ ]:
map_grid = None
btree = None
nA = None
nB = None
dist = None
idx = None

In [ ]:
gc.collect()

In [ ]:
map_grid_knn.head(3)

In [ ]:
map_grid_knn.to_csv('./midsave/map_grid_knn.csv', index = False)

### Point-to-polygon allocation of pixels to administrative areas

In [ ]:
map_grid = gpd.read_file('./midsave/map_grid_nopop.shp')

In [ ]:
len(map_grid)

In [ ]:
target = map_grid.shape[0]
chunk = 10**6

In [ ]:
map_grid_adm = pd.DataFrame()

for i in tqdm(range(0, target, chunk)):
    
    tmp = map_grid[0:chunk]
    
    map_grid = map_grid[~map_grid['grid_id'].isin(tmp['grid_id'])]
    
    map_grid_adm = map_grid_adm.append(gpd.tools.sjoin(tmp, map_commune[['MAP_ID', 'geometry']], how="left", op='intersects').drop(columns=['geometry', 'index_right']))

In [ ]:
len(map_grid_adm)

In [ ]:
len(map_grid)

In [ ]:
len(map_grid_adm.MAP_ID.unique())

In [ ]:
gc.collect()

Some pixels have been allocated to more than one commune

In [ ]:
map_grid_adm.drop_duplicates(subset = 'grid_id', keep = False, inplace = True)

Some pixels lay outside the country borders. Drop them.

In [ ]:
len(map_grid_adm)

In [ ]:
map_grid_adm = map_grid_adm.dropna()

In [ ]:
len(map_grid_adm)

In [ ]:
len(map_grid_adm.MAP_ID.unique())

In [ ]:
map_grid_nogeo = pd.read_csv('./midsave/map_grid_nogeo.csv')

In [ ]:
map_grid_adm = map_grid_adm.merge(map_grid_nogeo, on = 'grid_id', how = 'left')

In [ ]:
map_grid_adm[['MAP_ID', 'grid_id', 'pop_per_grid']].to_csv('./midsave/w_grid_adm.csv', index = False)
map_grid_adm[['MAP_ID', 'grid_id', 'pop_per_grid']].to_csv('./midsave/dem_proj_grid.csv', index = False)

# Compute spatial weights from grid

In [ ]:
map_grid_adm = pd.read_csv('./midsave/w_grid_adm.csv').astype(
        {'MAP_ID': 'uint16', 'grid_id': 'uint32', 'pop_per_grid': 'float64'})

In [ ]:
map_grid_knn = pd.read_csv('./midsave/map_grid_knn.csv').astype(
        {'grid_id': 'uint32', 'site_id': 'uint16', 'path_loss': 'float32'})

In [ ]:
map_grid_knn = map_grid_knn[map_grid_knn.grid_id.isin(map_grid_adm.grid_id) == True]

In [ ]:
map_grid_knn = map_grid_knn.merge(map_grid_adm[['MAP_ID', 'grid_id', 'pop_per_grid']], on = 'grid_id', how = 'left')

In [ ]:
map_grid_knn['received_signal_strength'] = 45 - map_grid_knn.path_loss

Dropping all connections that are below RSSI -110

In [ ]:
map_grid_knn = map_grid_knn[map_grid_knn.received_signal_strength > -110]

Implementing winner-takes-it-all aka. best server approach

In [ ]:
map_grid_knn.set_index('grid_id', inplace = True)
map_grid_knn['w_best_ant'] = 0
map_grid_knn.loc[map_grid_knn.groupby('grid_id')['received_signal_strength'].transform(max) == map_grid_knn.received_signal_strength, 'w_best_ant'] = 1
map_grid_knn.reset_index(inplace=True)

In [ ]:
df = map_grid_knn.groupby('grid_id')['w_best_ant'].sum().reset_index()

In [ ]:
for i in df[df.w_best_ant > 1].grid_id:
    
    if map_grid_knn[map_grid_knn.grid_id == i].w_best_ant.sum() > 1:
        
        map_grid_knn.loc[map_grid_knn.grid_id == i, 'w_best_ant'] = np.where(
            map_grid_knn[map_grid_knn.grid_id == i].w_best_ant == 1, 1/map_grid_knn[map_grid_knn.grid_id == i].w_best_ant.sum(), 0)
    
    else:
        continue

In [ ]:
map_grid_knn.set_index('site_id', inplace = True)
map_grid_knn['w_best_ant_pop'] = map_grid_knn.w_best_ant
map_grid_knn['w_best_ant_sum'] = map_grid_knn.groupby('site_id')['w_best_ant_pop'].sum()
map_grid_knn['w_best_site'] = map_grid_knn.w_best_ant_pop / map_grid_knn.w_best_ant_sum # might cause issues when w_best_ant_sum is 0
map_grid_knn.reset_index(inplace=True)

In [ ]:
map_grid_knn.groupby('site_id')['w_best_site'].sum().describe()

In [ ]:
map_grid_knn = map_grid_knn.drop(columns = {'w_best_ant_pop', 'w_best_ant_sum'})

In [ ]:
gc.collect()

Calculating inverse distance weights based on RSS measures.

In [ ]:
map_grid_knn['received_signal_strength_inverted'] = 1 / (map_grid_knn.received_signal_strength)**2

In [ ]:
map_grid_knn.set_index('grid_id', inplace = True)
map_grid_knn['rss_sum_inv'] = map_grid_knn.groupby('grid_id')['received_signal_strength_inverted'].sum()
map_grid_knn['w_knn_ant'] = map_grid_knn.received_signal_strength_inverted / map_grid_knn.rss_sum_inv
map_grid_knn.reset_index(inplace=True)

In [ ]:
map_grid_knn.set_index('site_id', inplace = True)
map_grid_knn['w_knn_ant_pop'] = map_grid_knn.w_knn_ant
map_grid_knn['w_knn_ant_sum'] = map_grid_knn.groupby('site_id')['w_knn_ant_pop'].sum()
map_grid_knn['w_knn_site'] = map_grid_knn.w_knn_ant_pop / map_grid_knn.w_knn_ant_sum
map_grid_knn.reset_index(inplace=True)

In [ ]:
map_grid_knn = map_grid_knn.drop(columns = {
    'received_signal_strength', 'received_signal_strength_inverted', 'rss_sum_inv', 'w_knn_ant', 'w_knn_ant_pop', 'w_knn_ant_sum'})

In [ ]:
gc.collect()

In [ ]:
map_grid_knn[['grid_id', 'site_id', 'path_loss', 'w_best_site', 'w_knn_site']].to_csv('./midsave/w_grid.csv', index = False)